In [542]:
import requests
import json
import pandas as pd
import time
import plotly.express as px
import os
import re
import boto3
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, Float, ForeignKey, JSON, MetaData, UniqueConstraint

In [310]:
# Initialize an empty dictionary to store sensitive data
sensitive_data = {}

# Read the config file and populate the dictionary
with open('config.txt', 'r') as file:
    for line in file:
        key, value = line.strip().split('=', 1)  # Split only on the first '='
        sensitive_data[key.strip()] = value.strip()

# Access your sensitive data

WeatherApiKey = sensitive_data.get('WeatherApiKey')
aws_access_key_id = sensitive_data.get('aws_access_key_id')
aws_secret_access_key = sensitive_data.get('aws_secret_access_key')
db_url = sensitive_data.get('db_url')

In [ ]:
cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

headers = {
    'User-Agent': 'MyApp (derop56705@adambra.com)'
}
df = pd.DataFrame(columns=['City', 'Coordinate'])

for city in cities:
    start_time = time.time()  # Record start time
    r = requests.get(f'https://nominatim.openstreetmap.org/search?city={city}&country=France&format=json', headers=headers)
    response = json.loads(r.text)
    if response:  # Check if the response is not empty
        coordinates = response[0]['lon'], response[0]['lat']
        df.loc[len(df)] = {'City': city, 'Coordinate': coordinates}
    else:
        print(f"No coordinates found for {city}")

    elapsed_time = time.time() - start_time  # Calculate elapsed time
    if elapsed_time < 1.5:
        time.sleep(1.5 - elapsed_time)  # Sleep for the remaining time / nominatim policy : No heavy uses (an absolute maximum of 1 request per second).


In [67]:
df.to_json('cities_coordinates.json', orient='records', indent=4)

In [27]:

# Separate coordinates into latitude and longitude columns
df['lon'] = df['Coordinate'].apply(lambda x: float(x[0]))
df['lat'] = df['Coordinate'].apply(lambda x: float(x[1]))



In [32]:
df.head()

,City,Coordinate,lon,lat
0,Mont Saint Michel,"(-1.511459954959514, 48.6359541)",-1.511460,48.635954
1,St Malo,"(-2.0260409, 48.649518)",-2.026041,48.649518
2,Bayeux,"(-0.7024738, 49.2764624)",-0.702474,49.276462
3,Le Havre,"(0.1079732, 49.4938975)",0.107973,49.493898
4,Rouen,"(1.0939658, 49.4404591)",1.093966,49.440459


In [46]:
# Create a scatter mapbox plot
fig = px.scatter_mapbox(df, lat="lat", lon="lon", hover_name="City", color_discrete_sequence=["fuchsia"], zoom=5, height=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [74]:
flattened_data = []
cities_data = df[['City', 'lat', 'lon']].to_dict(orient='records')

for city_data in cities_data:
    start_time = time.time()  # Record start time
    params = {
        'lat': city_data["lat"],
        'lon': city_data["lon"],
        'appid': WeatherApiKey,
        'units': 'metric'
    }
    response = requests.get(f'https://api.openweathermap.org/data/2.5/forecast', params=params).json()
    if 'list' in response:  # Check if the response contains 'list'
        flattened_json = [
        {
            'city': city_data["City"],
            'lat': city_data["lat"],
            'lon': city_data["lon"],
            'datetime': item['dt_txt'],
            'temp': item['main']['temp'],
            'feels_like': item['main']['feels_like'],
            'temp_min': item['main']['temp_min'],
            'temp_max': item['main']['temp_max'],
            'pressure': item['main']['pressure'],
            'sea_level': item['main']['sea_level'],
            'grnd_level': item['main']['grnd_level'],
            'humidity': item['main']['humidity'],
            'weather_main': item['weather'][0]['main'],
            'weather_description': item['weather'][0]['description'],
            'weather_icon': item['weather'][0]['icon'],
            'clouds': item['clouds']['all'],
            'wind_speed': item['wind']['speed'],
            'wind_deg': item['wind']['deg'],
            'wind_gust': item['wind']['gust'],
            'visibility': item['visibility'],
            'pop': item['pop'],
            'rain': item.get('rain', {}).get('3h', 0),
            'snow': item.get('snow', {}).get('3h', 0),
            'pod': item['sys']['pod']
        } for item in response['list']]

        flattened_data.extend(flattened_json)  # Extend the list directly
    else:
        print(f"No weather forecast found for {city_data['City']}")

    elapsed_time = time.time() - start_time  # Calculate elapsed time
    if elapsed_time < 1.5:
        time.sleep(1.5 - elapsed_time)  # Sleep for the remaining time / openweathermap Free collection: 60 calls/minute

weatherdf = pd.DataFrame(flattened_data)


weatherdf.head()

,city,lat,lon,datetime,temp,feels_like,temp_min,temp_max,pressure,sea_level,...,weather_icon,clouds,wind_speed,wind_deg,wind_gust,visibility,pop,rain,snow,pod
0,Mont Saint Michel,48.635954,-1.51146,2024-10-11 00:00:00,10.17,9.72,9.12,10.17,1013,1013,...,10n,70,2.52,68,2.82,10000,0.25,0.17,0,n
1,Mont Saint Michel,48.635954,-1.51146,2024-10-11 03:00:00,9.55,8.49,8.32,9.55,1013,1013,...,03n,47,2.22,103,2.26,10000,0.00,0.00,0,n
2,Mont Saint Michel,48.635954,-1.51146,2024-10-11 06:00:00,8.64,7.23,7.87,8.64,1014,1014,...,02n,23,2.48,105,2.76,10000,0.00,0.00,0,n
3,Mont Saint Michel,48.635954,-1.51146,2024-10-11 09:00:00,11.77,11.04,11.77,11.77,1016,1016,...,01d,3,2.58,114,4.20,10000,0.00,0.00,0,d
4,Mont Saint Michel,48.635954,-1.51146,2024-10-11 12:00:00,15.35,14.48,15.35,15.35,1015,1015,...,01d,4,3.89,108,4.30,10000,0.00,0.00,0,d


In [219]:
weatherdf.to_json('cities_weather_rating.json', orient='records', indent=4)

In [218]:

# Normalize factors (feels_like, rain, snow) with highest importance
weights = {
    'feels_like': 4,
    'humidity': 2,
    'clouds': 3,
    'wind_speed': 3,
    'visibility': 2,
    'rain': 5,
    'snow': 5
}

def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

weatherdf['feels_like_norm'] = normalize(abs(weatherdf['feels_like'] - 25)) * weights['feels_like']
weatherdf['humidity_norm'] = normalize(abs(weatherdf['humidity'] - 50)) * weights['humidity']
weatherdf['clouds_norm'] = normalize(abs(weatherdf['clouds'] - 0)) * weights['clouds']
weatherdf['wind_speed_norm'] = normalize(abs(weatherdf['wind_speed'] - 0)) * weights['wind_speed']
weatherdf['visibility_norm'] = normalize(abs(weatherdf['visibility'] - 10000)) * weights['visibility']
weatherdf['rain_norm'] = normalize(abs(weatherdf['rain'] - 0)) * weights['rain']
weatherdf['snow_norm'] = normalize(abs(weatherdf['snow'] - 0)) * weights['snow']

# Compute the Rating
weatherdf['weather_rating'] = weatherdf[
    ['feels_like_norm', 'humidity_norm', 'clouds_norm', 
     'wind_speed_norm', 'visibility_norm', 'rain_norm', 'snow_norm']
].sum(axis=1)



In [217]:
weatherdf = pd.read_json('src/cities_weather.json')

In [299]:
# Group by city and calculate the median rating
city_grouped = weatherdf.groupby('city')['weather_rating'].median().reset_index()
# Include lat and lon by taking the first occurrence (since they are constant per city)
city_grouped['lat'] = weatherdf.groupby('city')['lat'].first().values
city_grouped['lon'] = weatherdf.groupby('city')['lon'].first().values
# Get the top 7 cities with the best median weather rating
top_5_cities = city_grouped.nsmallest(5, 'weather_rating')

print(top_5_cities)


                        city  weather_rating        lat       lon
1            Aix en Provence        4.071522  43.529842  5.447474
24                 Marseille        4.211053  43.296174  5.369953
12                    Cassis        4.375003  43.214036  5.539632
27                     Nimes        4.533238  43.837425  4.360069
30  Saintes Maries de la mer        4.900959  43.451592  4.427720


In [223]:
weatherdf['city_id'] = pd.factorize(weatherdf['city'])[0]

In [224]:
weatherdf.to_csv('src/cities_weather_rating.csv')

In [227]:
fig = px.scatter_mapbox(city_grouped, lat="lat", lon="lon", hover_name="city", color="weather_rating", color_continuous_scale="temps",range_color=[city_grouped['weather_rating'].min(), city_grouped['weather_rating'].max() ], zoom=5, height=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_traces(marker=dict(size=15))  
fig.show()
fig.write_html("src/cities_map_plotly.html")

In [91]:
!python src/BookingSpider.py

2024-10-11 20:10:50 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-10-11 20:10:50 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 24.0.0 (OpenSSL 3.0.14 4 Jun 2024), cryptography 42.0.5, Platform Windows-11-10.0.22631-SP0
2024-10-11 20:10:51 [scrapy.addons] INFO: Enabled addons:
[]
2024-10-11 20:10:51 [py.warnings] WARNING: c:\Users\ad98\anaconda3\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the docum

In [52]:
os.chdir("src")

In [411]:
data = json.load(open('BookingInfo.json', 'r'))
df = pd.DataFrame(data)
urlList = df['url'].tolist()

urlList

['https://www.booking.com/hotel/fr/hotel-saint-aubert.fr.html',
 'https://www.booking.com/hotel/fr/mont-saint-michel.fr.html',
 'https://www.booking.com/hotel/fr/le-relais-saint-michel.fr.html',
 'https://www.booking.com/hotel/fr/la-mere-poulard.fr.html',
 'https://www.booking.com/hotel/fr/les-terrasses-poulard.fr.html',
 'https://www.booking.com/hotel/fr/la-vieille-auberge-le-mont-saint-michel.fr.html',
 'https://www.booking.com/hotel/fr/le-relais-du-roy.fr.html',
 'https://www.booking.com/hotel/fr/vert.fr.html',
 'https://www.booking.com/hotel/fr/le-mouton-blanc-le-mont-saint-michel.fr.html',
 'https://www.booking.com/hotel/fr/ha-el-la-croix-blanche.fr.html',
 'https://www.booking.com/hotel/fr/hotel-gabriel.fr.html',
 'https://www.booking.com/hotel/fr/auberge-saint-pierre.fr.html',
 'https://www.booking.com/hotel/fr/auberge-de-la-baie.fr.html',
 'https://www.booking.com/hotel/fr/ermitage-mont-saint-michel-beauvoir.fr.html',
 'https://www.booking.com/hotel/fr/vacanceole-les-chambres-d

In [3]:
!scrapy startproject hotel_data_scraping

New Scrapy project 'hotel_data_scraping', using template directory 'C:\Users\ad98\anaconda3\Lib\site-packages\scrapy\templates\project', created in:
    C:\Users\ad98\Desktop\Jedha\kayak\src\hotel_data_scraping

You can start your first spider with:
    cd hotel_data_scraping
    scrapy genspider example example.com


In [53]:
os.chdir("hotel_data_scraping")

In [11]:
with open("Hotel_url_list.txt", 'w') as file:
    for url in urlList:
        file.write(url + '\n')

In [36]:
!scrapy crawl hotels

2024-10-12 18:38:50 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: hotel_data_scraping)
2024-10-12 18:38:50 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 24.0.0 (OpenSSL 3.0.14 4 Jun 2024), cryptography 42.0.5, Platform Windows-11-10.0.22631-SP0
2024-10-12 18:38:50 [scrapy.addons] INFO: Enabled addons:
[]
2024-10-12 18:38:50 [scrapy.extensions.telnet] INFO: Telnet Password: 14c21db588f3c1d1
2024-10-12 18:38:50 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.throttle.AutoThrottle']
2024-10-12 18:38:50 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'AUTOTHROTTLE_START_DELAY': 1

In [413]:
df_reduced = df.groupby('city').head(20).reset_index(drop=True)
urlList = df_reduced['url'].tolist()


In [6]:
with open("Hotel_url_list_reduced.txt", 'w') as file:
    for url in urlList:
        file.write(url + '\n')

In [198]:
!scrapy crawl hotels

2024-10-14 15:17:09 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: hotel_data_scraping)
2024-10-14 15:17:10 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 24.0.0 (OpenSSL 3.0.14 4 Jun 2024), cryptography 42.0.5, Platform Windows-11-10.0.22631-SP0
2024-10-14 15:17:10 [scrapy.addons] INFO: Enabled addons:
[]
2024-10-14 15:17:10 [scrapy.extensions.telnet] INFO: Telnet Password: e65b9fab8aacac82
2024-10-14 15:17:10 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.throttle.AutoThrottle']
2024-10-14 15:17:10 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'AUTOTHROTTLE_START_DELAY': 1

In [426]:
data = json.load(open('results/hotel_data.json', 'r'))
df = pd.DataFrame(data)

In [427]:
df['address'] = df['address'].str.replace("\n", "")

In [428]:
def clean_street(street):
    # Start by isolating the relevant part of the street address before any " - "
    formatted_street_name = street.split(" - ")[0]
    
    # Check if the address does not contain the word 'angle' (case-insensitive)
    if not re.search(r'angle', formatted_street_name, re.IGNORECASE):
        # If there are multiple streets or numbers separated by " et ", retain only 1
        formatted_street_name = formatted_street_name.split(" et ")[0] if len(formatted_street_name.split(" et ")[0]) > 3 else formatted_street_name.split(" et ")[1]
        formatted_street_name = formatted_street_name.split(" Et ")[0] if len(formatted_street_name.split(" Et ")[0]) > 3 else formatted_street_name.split(" Et ")[1]

    # Check for a street number at the start of the cleaned text
    match = re.match(r'(^\d+)', formatted_street_name)
    if match:
        # Extract the street number and strip any surrounding whitespace
        number = match.group(0).strip()
        
        # Remove the number from the cleaned text to isolate the street name
        streetName = re.sub(r'^\d+(-\d+)?|(/\d+)?(-\d+)?', '', formatted_street_name).strip(',').strip()  # Also strip any trailing commas
        formatted_street_name = number + " " + streetName  # Recombine the number and street name

    # Remove any additional information after a comma, if present
    formatted_street_name = formatted_street_name.split(",")[0].strip()

    return formatted_street_name


# def TryGetCoord(params) :
#     start_time = time.time()  # Record start time
#     r = requests.get(f'https://nominatim.openstreetmap.org/search', headers=headers, params=params)
#     if r.status_code != 200 :
#         print(r)
#     elapsed_time = time.time() - start_time  # Calculate elapsed time
#     if elapsed_time < 1.2:
#         time.sleep(1.2 - elapsed_time)  # Sleep for the remaining time / nominatim policy : No heavy uses (an absolute maximum of 1 request per second).
#     return r


def GetCoord(row) : 
    address = row['address']
    # hotel_name = row['hotel_name'].strip()
    street, postalCode_city, country = address.rsplit(',', 2)
    postalCode , city = postalCode_city.strip().split(' ', 1)
    street = clean_street(street)
    postalCode = postalCode.strip()
    city = city.strip()
    country = country.strip()

    # params_list = []


    # # Check if the street starts with a number
    # match = re.match(r'^\d', street)
    
    # if match:
    #     params_list.append({'street': street, 'postalCode' : postalCode, 'country' : country, 'format': 'json'})
    #     params_list.append({'q': f'{street} {city} {country}', 'format': 'json'})

    # params_list.append({'amenity': hotel_name, 'postalCode' : postalCode, 'country' : country, 'format': 'json'})
    # params_list.append({'q': f'{hotel_name} {city} {country}', 'format': 'json'})

    # if not match:
    #     params_list.append({'street': street, 'postalCode' : postalCode, 'country' : country, 'format': 'json'})
    #     params_list.append({'q': f'{street} {city} {country}', 'format': 'json'})
    

    # if " - " in hotel_name:
    #     hotel_name_split = hotel_name.split(" - ", 1)[0]
    #     params_list.append({'amenity': hotel_name_split, 'postalCode':postalCode, 'country' : country, 'format': 'json'})
    #     params_list.append({'q': f'{hotel_name_split} {city} {country}', 'format': 'json'})

    # if ", " in hotel_name:
    #     hotel_name_split = hotel_name.split(", ", 1)[0]
    #     params_list.append({'amenity': hotel_name_split, 'postalCode':postalCode, 'country' : country, 'format': 'json'})
    #     params_list.append({'q': f'{hotel_name_split} {city} {country}', 'format': 'json'})

    # if " et " in hotel_name.lower():
    #     hotel_name_split = hotel_name.lower().split(" et ", 1)[0]
    #     params_list.append({'amenity': hotel_name_split, 'postalCode':postalCode, 'country' : country, 'format': 'json'})
    #     params_list.append({'q': f'{hotel_name_split} {city} {country}', 'format': 'json'})

    # if "rn " in hotel_name.lower():
    #     street_name_modified = hotel_name.lower().replace("rn ", "route nationale")
    #     params_list.append({'street': street_name_modified, 'postalCode':postalCode, 'country' : country, 'format': 'json'})
    #     params_list.append({'q': f'{street_name_modified} {city} {country}', 'format': 'json'})
    # #params_list.append({'city': city, 'country': country, 'format': 'json'})

    # lon = None
    # lat = None

    # for params in params_list:
    #     r = TryGetCoord(params)
    #     response = json.loads(r.text) if r.status_code == 200 else None
    #     if response:
    #         lon = response[0]['lon']
    #         lat = response[0]['lat']
    #         break
    #     print(f"Failed to find coordinates for {params}")

    # if lon is None or lat is None:
    #     print(f"No coordinates found for {hotel_name}")

    return street, postalCode, city, country

df[['Street', 'PostalCode', 'City', 'Country']] = pd.DataFrame(df.apply(GetCoord, axis=1).tolist(), columns=['Street', 'PostalCode', 'City', 'Country'])




In [429]:
# Convert lat and lon columns to float
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)


In [430]:

# Create a scatter mapbox plot
fig = px.scatter_mapbox(df, lat="lat", lon="lon", hover_name="hotel_name",hover_data=["address"], color_discrete_sequence=["fuchsia"], zoom=5, height=800)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


In [ ]:
os.chdir("..")

In [461]:
df_urls = pd.read_json('BookingInfo.json')

In [462]:
df_weather = weatherdf.groupby('city').median(numeric_only=True).reset_index()

In [463]:
df_hotels = df

In [464]:
df_hotels = df_hotels.rename(columns={'City': 'hotel_city'})
df_hotels = df_hotels.rename(columns={'lat': 'hotel_lat'})
df_hotels = df_hotels.rename(columns={'lon': 'hotel_lon'})

In [465]:
df_hotels_urls = pd.merge(df_hotels, df_urls, on='url', how='left')
df_final = pd.merge(df_hotels_urls, df_weather, on='city', how='left').copy()
df_final = df_final.drop(columns=['Result name'])

In [466]:
# Create a scatter mapbox plot
fig = px.scatter_mapbox(df_final[df_final['city'].isin(top_5_cities['city'])], lat="hotel_lat", lon="hotel_lon", hover_name="hotel_name",hover_data=["address"], color_discrete_sequence=["fuchsia"], zoom=5, height=800)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
fig.write_html("hotels_in_best_weather_cities.html")

In [467]:
df_final.to_csv('france_top_cities_weather_hotels.csv')

In [292]:
session = boto3.Session(aws_access_key_id=aws_access_key_id, 
                        aws_secret_access_key=aws_secret_access_key)

In [293]:
s3 = session.resource("s3")

In [296]:
bucket =  s3.create_bucket(Bucket="bucket-kayak-project-56484212", CreateBucketConfiguration={'LocationConstraint': "eu-west-3"})

In [468]:
put_object = bucket.put_object(Key="france_top_cities_weather_hotels.csv", Body=df_final.to_csv())

In [469]:
df_final = df_final.dropna(subset=['city']) # drop hotels if couldn't link to a city

In [470]:
missing_values = df_final.isnull().sum()

fig = px.bar(missing_values[missing_values > 0])
fig.show()

In [471]:
df_final.fillna({'users_global_rating': '0.0', 'snow_norm': 0}, inplace=True)

In [472]:
df_final.isnull().sum().max()

0

In [ ]:
# Function to get the most common type in a column
def most_common_dtype(series):
    return series.apply(type).mode()[0]

# Determine the expected data types for each column
expected_dtypes = {column: most_common_dtype(df_final[column]) for column in df_final.columns}

# Apply the check for incorrect types
for column, expected_type in expected_dtypes.items():
    actual_types = df_final[column].apply(type)
    wrong_type_indices = actual_types[actual_types != expected_type].index
    if not wrong_type_indices.empty:
        print(f"Column '{column}' has wrong data types at indices: {list(wrong_type_indices)}")

print("Expected data types for each column:")
print(expected_dtypes)



Expected data types for each column:
{'hotel_name': <class 'str'>, 'url': <class 'str'>, 'hotel_lat': <class 'float'>, 'hotel_lon': <class 'float'>, 'address': <class 'str'>, 'users_global_rating': <class 'str'>, 'hotel_desc': <class 'str'>, 'key_features': <class 'list'>, 'users_category_rating': <class 'dict'>, 'stars': <class 'int'>, 'amenities': <class 'dict'>, 'Street': <class 'str'>, 'PostalCode': <class 'str'>, 'hotel_city': <class 'str'>, 'Country': <class 'str'>, 'city': <class 'str'>, 'lat': <class 'float'>, 'lon': <class 'float'>, 'temp': <class 'float'>, 'feels_like': <class 'float'>, 'temp_min': <class 'float'>, 'temp_max': <class 'float'>, 'pressure': <class 'float'>, 'sea_level': <class 'float'>, 'grnd_level': <class 'float'>, 'humidity': <class 'float'>, 'clouds': <class 'float'>, 'wind_speed': <class 'float'>, 'wind_deg': <class 'float'>, 'wind_gust': <class 'float'>, 'visibility': <class 'float'>, 'pop': <class 'float'>, 'rain': <class 'float'>, 'snow': <class 'float'

In [480]:
df_final['city_id'] = df_final['city_id'].astype(int)
df_final['users_global_rating'] = df_final['users_global_rating'].astype(float)

In [474]:
# Drop unnecessary columns
df_final.drop(columns=['feels_like_norm', 'humidity_norm', 'clouds_norm', 'wind_speed_norm', 'visibility_norm', 'rain_norm', 'snow_norm'], inplace=True)

In [475]:
df_final.columns

Index(['hotel_name', 'url', 'hotel_lat', 'hotel_lon', 'address',
       'users_global_rating', 'hotel_desc', 'key_features',
       'users_category_rating', 'stars', 'amenities', 'Street', 'PostalCode',
       'hotel_city', 'Country', 'city', 'lat', 'lon', 'temp', 'feels_like',
       'temp_min', 'temp_max', 'pressure', 'sea_level', 'grnd_level',
       'humidity', 'clouds', 'wind_speed', 'wind_deg', 'wind_gust',
       'visibility', 'pop', 'rain', 'snow', 'weather_rating', 'city_id'],
      dtype='object')

In [513]:
engine = create_engine(db_url, echo=True)

In [544]:

metadata = MetaData()

# Define the cities table
cities = Table('cities', metadata,
    Column('city_id', Integer, primary_key=True, autoincrement=True),
    Column('city', String),
    Column('lat', Float),
    Column('lon', Float),
    Column('weather_rating', Float),
    Column('temp', Float),
    Column('feels_like', Float),
    Column('humidity', Float),
    Column('clouds', Float),
    Column('wind_speed', Float),
    Column('wind_deg', Float),
    Column('wind_gust', Float),
    Column('visibility', Float)
)

# Define the hotels table
hotels = Table('hotels', metadata,
    Column('hotel_id', Integer, primary_key=True, autoincrement=True),
    Column('hotel_name', String),
    Column('city_id', Integer, ForeignKey('cities.city_id')),
    Column('address', String),
    Column('users_global_rating', Float),
    Column('hotel_desc', String),
    Column('key_features', JSON),
    Column('users_category_rating', JSON),
    Column('stars', Float),
    Column('amenities', JSON),
    Column('Street', String),
    Column('PostalCode', String),
    Column('Country', String),
    Column('hotel_lat', Float),
    Column('hotel_lon', Float),
    Column('url', String)
)

# Create all tables in the database
metadata.create_all(engine)



2024-10-15 01:21:07,433 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-15 01:21:07,434 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2024-10-15 01:21:07,434 INFO sqlalchemy.engine.Engine [cached since 2834s ago] {'table_name': 'cities', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2024-10-15 01:21:07,461 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_c

In [545]:
# Create a DataFrame for the cities table
cities_df = df_final[['city_id','city', 'lat', 'lon', 'weather_rating', 'temp', 'feels_like', 'humidity', 'clouds', 'wind_speed','wind_deg','wind_gust','visibility']].drop_duplicates()


In [546]:
# Append the DataFrame to the existing table
cities_df.to_sql('cities', con=engine, index=False, if_exists='append')

2024-10-15 01:21:23,167 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-15 01:21:23,168 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2024-10-15 01:21:23,168 INFO sqlalchemy.engine.Engine [cached since 2850s ago] {'table_name': 'cities', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2024-10-15 01:21:23,195 INFO sqlalchemy.engine.Engine INSERT INTO cities (city_id, city, lat, lon, weather_rating, temp, feels_like, humidity, clouds, wind_speed, wind_deg, wind_gust, visibility) VALUES (%(city_id__0)s, %(city__0)s,

35

In [547]:
hotels_df = df_final[['hotel_name','city_id', 'address', 'users_global_rating', 'hotel_desc', 'key_features', 'users_category_rating', 'stars', 'amenities', 'PostalCode','Country','hotel_lat','hotel_lon','url']].copy()
hotels_df['hotel_id'] = range(1, len(hotels_df) + 1)

In [548]:
hotels_df['key_features'] = hotels_df['key_features'].apply(json.dumps)
hotels_df['users_category_rating'] = hotels_df['users_category_rating'].apply(json.dumps)
hotels_df['amenities'] = hotels_df['amenities'].apply(json.dumps)

hotels_df['users_global_rating'] = hotels_df['users_global_rating'].str.replace(',', '.').astype(float)
#hotels_df['hotel_desc'] = hotels_df['hotel_desc'].apply(lambda x: x.encode('utf-8').decode('utf-8'))

In [549]:
hotels_df.to_sql('hotels', con=engine, index=False, if_exists='append')

2024-10-15 01:21:36,246 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-15 01:21:36,247 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2024-10-15 01:21:36,248 INFO sqlalchemy.engine.Engine [cached since 2863s ago] {'table_name': 'hotels', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2024-10-15 01:21:36,281 INFO sqlalchemy.engine.Engine INSERT INTO hotels (hotel_name, city_id, address, users_global_rating, hotel_desc, key_features, users_category_rating, stars, amenities, "PostalCode", "Country", hotel_lat, hote

769